In [ ]:
"""Import all the necessary packages"""
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


"""Test for GPU or CPU devices"""
print("Device: \n", tf.config.experimental.list_physical_devices())
print(tf.__version__)
print(tf.test.is_built_with_cuda())

"""Contrast Limited Adaptive Histogram Equalization (CLAHE) Method for preprocessing"""

def clahe_function(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    clahe_img = clahe.apply(l)
    updated_lab_img2 = cv2.merge((clahe_img, a, b))
    # Convert LAB image back to color (RGB)
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
    return CLAHE_img

"""Funcntion for loading images"""

images = []
def load_images_from_folder(folder):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        #img = cv2.resize(img, (224, 224))
        img = clahe_function(img)
        img = img/255.0
        img = cv2.resize(img, (224, 224))
        if img is not None:
            images.append(img)
    return images

covid = r"D:\\Research Related W"
ork\\brain cancer\\Dataset\\1s Dataset preprocess\\copy\\glioma_tumor cl"# directory of covid image
#non_covid = r"/content/drive/MyDrive/Colab Notebooks/image pp/ADI"   # directory of non-covid image

load_images_from_folder(covid)
#load_images_from_folder(non_covid)

y = np.ones(4000)  #labeling non-covid images as 1
y = np.append(y, np.zeros(5000))   #labeling covid images as 0
y = list(y)
c = list(zip(images, y))

#reshuffling all the images along with their labels

random.shuffle(c)
images, y = zip(*c)
del c  #For Memory Efficiency
images = np.array(images)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.15, stratify= y, random_state =2)

"""Our proposed Convolutional Model for extracting features"""

model = tf.keras.Sequential()
#1st conv layer
model.add(tf.keras.layers.Conv2D(8, 3, input_shape=(224, 224, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
#2nd conv layer
model.add(tf.keras.layers.Conv2D(16, 3, padding="valid"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
#3rd conv layer (from here model gives good result)
model.add(tf.keras.layers.Conv2D(32, 3, padding="valid"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
#4th conv layer
model.add(tf.keras.layers.Conv2D(64, 3, padding="valid"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
#Flatten Layer
model.add(tf.keras.layers.Flatten())
#Dense Layer 1
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
#Dense Layer 2
model.add(tf.keras.layers.Dense(100,name ='feature_denseee')) #100 Prominant Features are Extracted From This Layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
#output Dense Layer
model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Activation('softmax'))
adam = tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer='adam')
model.summary()

#CNN Model Trained
with tf.device('/GPU:0'):
    history = model.fit(X_train, y_train, batch_size= 32, epochs=15, verbose=1, validation_data=(X_test, y_test))

""" Save the model """
model.save("D:\\Research Related Work\\brain cancer\\Dataset\\1s Dataset preprocess\\Normal\\cov.h5")

""" Load the model """
model = tf.keras.models.load_model("D:\\Research Related Work\\brain cancer\\Dataset\\1s Dataset preprocess\\Normal\\cov.h5")
intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('feature_denseee').output)
intermediate_layer_model.summary()

"""Features Extraction From All Images"""
feature_engg_data = intermediate_layer_model.predict(images)
feature_engg_data = pd.DataFrame(feature_engg_data)

"""Save The Features"""
#feature_engg_data.to_csv("E:\Research Related Work\Zahid sir\image pp-20221112T121944Z-001\data/feature.csv")
#features = pd.read_csv("E:\Research Related Work\Zahid sir\image pp-20221112T121944Z-001\data/feature.csv")

"""Normalization of The Features"""
from sklearn.preprocessing import StandardScaler
x = feature_engg_data.loc[:, feature_engg_data.columns].values
x = StandardScaler().fit_transform(x)

"""Splitting the Data into Training & Testing Set"""
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, stratify= y, random_state =0)


#feature_engg_data1 = pd.DataFrame(y)
#feature_engg_data1.to_csv("/content/drive/MyDrive/Colab Notebooks/Zahid sir Work/Feature Extract/Cnn 100 feature extract/featurey.csv")



In [ ]:
#feature_engg_data = intermediate_layer_model.predict(x)
feature_engg_data = pd.DataFrame(x)
feature_engg_data.to_csv("D:\\Research Related Work\\brain cancer\\Dataset\\1s Dataset preprocess\\copy\\glioma_tumor m s\\glioma_tumo.csv")


In [ ]:
!conda activate your_environment_name
